# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `RequestAccepted` から **友だち数が最大の人**とその**友だち数**を求める（フレンドは無向）
* 入力: `RequestAccepted(requester_id INT, accepter_id INT, accept_date DATE)`
  ※ `(requester_id, accepter_id)` がユニーク
* 出力: `id INT, num INT`

  * 同率最大があれば全員を返してよい

## 2) 最適解（単一クエリ）

> 方向別に**先に集計**してから合算し、`DENSE_RANK` で最大のみ抽出。中間行を増やさないのでスケールしやすいです。

```sql
WITH deg_parts AS (
  -- 方向別に一度だけ集計
  SELECT requester_id AS id, COUNT(*) AS cnt
  FROM RequestAccepted
  GROUP BY requester_id
  UNION ALL
  SELECT accepter_id  AS id, COUNT(*) AS cnt
  FROM RequestAccepted
  GROUP BY accepter_id
),
deg AS (
  -- 片方向集計の合算 = 各ユーザーの友だち数
  SELECT id, SUM(cnt) AS num
  FROM deg_parts
  GROUP BY id
),
ranked AS (
  SELECT
    id,
    num,
    DENSE_RANK() OVER (ORDER BY num DESC) AS rnk
  FROM deg
)
SELECT id, num
FROM ranked
WHERE rnk = 1;

Runtime 184 ms
Beats 49.75%

```

### 代替（重複エッジに堅牢：`(a,b)` と `(b,a)` が**両方**存在し得る場合）

> 実運用を強く意識するなら、**無向化＋一意化**してから次数カウントを。

```sql
WITH uniq_pairs AS (
  SELECT
    LEAST(requester_id, accepter_id)  AS u,
    GREATEST(requester_id, accepter_id) AS v
  FROM RequestAccepted
  GROUP BY LEAST(requester_id, accepter_id), GREATEST(requester_id, accepter_id)
),
deg AS (
  SELECT u AS id, COUNT(*) AS num FROM uniq_pairs GROUP BY u
  UNION ALL
  SELECT v AS id, COUNT(*) AS num FROM uniq_pairs GROUP BY v
),
sumdeg AS (
  SELECT id, SUM(num) AS num FROM deg GROUP BY id
)
SELECT id, num
FROM (
  SELECT id, num, DENSE_RANK() OVER (ORDER BY num DESC) AS rnk
  FROM sumdeg
) s
WHERE rnk = 1;

Runtime 177 ms
Beats 65.74%

```

### 参考（単一勝者が保証されるなら最短形）

```sql
WITH deg_parts AS (
  SELECT requester_id AS id, COUNT(*) AS cnt FROM RequestAccepted GROUP BY requester_id
  UNION ALL
  SELECT accepter_id  AS id, COUNT(*) AS cnt FROM RequestAccepted GROUP BY accepter_id
),
deg AS (
  SELECT id, SUM(cnt) AS num FROM deg_parts GROUP BY id
)
SELECT id, num
FROM deg
ORDER BY num DESC
LIMIT 1;

Runtime 175 ms
Beats 71.79%

```

## 3) 要点解説

* **中間行の抑制**：`UNION ALL` で 2N 行の「両方向エッジ」を作るより、**方向別で先に集計→合算**が軽量になりやすい
* **同率最大対応**：`DENSE_RANK() = 1` で自然に複数行を返せる
* **重複エッジ耐性**：実データでは `(a,b)` と `(b,a)` 共存があり得るため、堅牢性を求めるなら `LEAST/GREATEST` で無向ペアを正規化してから集計
* **演算子クラス等は不要**：単純な集計と順位のみ。列は整数で OK

## 4) 計算量（概算）

* 方向別 `GROUP BY`：**O(N)**〜**O(N log N)**
* 合算 `GROUP BY`：**O(U)**〜**O(U log U)**（`U` はユーザー数）
* `DENSE_RANK`：**O(U log U)**（内部ソート相当）

> いずれも入出力に対して線形〜準線形で、`edges` 全展開型より実務では有利なことが多いです。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[RequestAccepted] --> B[requester 側で集計]
  A --> C[accepter 側で集計]
  B --> D[合算して次数 num]
  C --> D
  D --> E[ウィンドウで順位付け]
  E --> F[順位1のみ抽出 id,num]
```

# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `RequestAccepted` から **友だち数が最大の人**とその**友だち数**を求める（友だちは無向）
* 入力: `RequestAccepted(requester_id INT, accepter_id INT, accept_date DATE)`（`(requester_id, accepter_id)` が一意）
* 出力: `id INT, num INT`（同率最大は全員返してよい）

## 2) 最適解（単一クエリ）

> あなたの解は正しいです。**さらに速く・一度のスキャンで済ませる**ために、`GROUPING SETS` を使って requester/accepter の片方向集計を**1 パス**で出し、その後合算します。
> 同率最大は `FETCH FIRST 1 ROW WITH TIES` で素直に対応（ウィンドウ関数不要）。

```sql
WITH parts AS (
  -- 1 回のスキャンで requester 側・accepter 側の件数を同時に作る
  SELECT COALESCE(requester_id, accepter_id) AS id, COUNT(*) AS cnt
  FROM RequestAccepted
  GROUP BY GROUPING SETS ((requester_id), (accepter_id))
),
deg AS (
  -- 片方向集計の合算 = 各ユーザーの友だち数
  SELECT id, SUM(cnt) AS num
  FROM parts
  GROUP BY id
)
SELECT id, num
FROM deg
ORDER BY num DESC
FETCH FIRST 1 ROW WITH TIES;

Runtime 201 ms
Beats 26.18%

```

* ポイント

  * あなたの `UNION ALL` 版はテーブルを**実質 2 回**集計しますが、`GROUPING SETS` は**1 回**で両方の集計を生成します。
  * 「最大が 1 人だけ」の問題でも、この形は**同率最大**を自動で拾える（`WITH TIES`）。

### 実運用により堅牢（相互重複 `(a,b)` と `(b,a)` が両在する場合）

```sql
WITH uniq_pairs AS (  -- 無向化して一意化
  SELECT DISTINCT
         LEAST(requester_id, accepter_id)  AS u,
         GREATEST(requester_id, accepter_id) AS v
  FROM RequestAccepted
),
deg AS (              -- 各端点を 1 回ずつ数える（1 ステップ集計）
  SELECT id, COUNT(*) AS num
  FROM uniq_pairs
  CROSS JOIN LATERAL (VALUES (u), (v)) AS t(id)
  GROUP BY id
)
SELECT id, num
FROM deg
ORDER BY num DESC
FETCH FIRST 1 ROW WITH TIES;

Runtime 180 ms
Beats 58.45%

```

* `LATERAL (VALUES ...)` で「各エッジを二端点に展開 → その場で集計」。
  `UNION ALL` で 2 回 `GROUP BY` するより、**一度の集計**で済むため計画がシンプルになりやすいです。

## 3) 要点解説

* **I/O 削減が最重要**：`GROUPING SETS` で requester/accepter を**一括集計** → 合算、で無駄を減らす
* **同率最大の取り回し**：`ORDER BY num DESC FETCH FIRST 1 ROW WITH TIES` が最短・明快
* **相互重複対策**：`LEAST/GREATEST` で無向ペアを正規化し、**単一集計**で次数を数えるのが堅牢

## 4) 計算量（概算）

* `GROUPING SETS` による二方向同時集計：**O(N)**～**O(N log N)**
* 合算 `GROUP BY`（ユーザー数 U）：**O(U)**～**O(U log U)**
* `FETCH ... WITH TIES`：**O(U log U)**（最大順位の範囲のみ抽出）

> 先に requester / accepter を別々に `GROUP BY` してから `UNION ALL` するより、**1 スキャン + 2 段集約**（`GROUPING SETS` → 合算）の方が実務で伸びやすいです。

---

### 追加の実務チューニング（任意）

* 片方向集計が多いワークロードなら、**両列に B-Tree**

  ```sql
  CREATE INDEX ON RequestAccepted (requester_id);
  CREATE INDEX ON RequestAccepted (accepter_id);
  ```

  大規模なら **並列集計**（`max_parallel_workers_per_gather`）や `enable_hashagg` 有効化で **HashAggregate** を選ばせると効くことが多いです。

---

> まとめ：あなたの 175–184ms は十分速いです。上記 **`GROUPING SETS` + `WITH TIES`**、または **無向正規化 + 単一集計** は、さらにプランが素直になり、データ量が増えるほど優位になりやすい最適化です。
